# AI Applications Project: Report Generation

In [1]:
#Importing the generative ai
%pip install -U -q "google-genai>=1.4.0"

Note: you may need to restart the kernel to use updated packages.


### Setting up API Key

In [ ]:
from google import genai

GOOGLE_API_KEY = input("Enter your Google API key:")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro","gemini-2.0-flash"] {"allow-input":true, isTemplate: true}

# Data

## Database Integration

In [8]:
import numpy as np
import pandas as pd
import os
import sys

sys.path.append('../../')

from Database.db import SessionLocal
from Database_Table import Inventory, Order

def getDbContent():
    session = SessionLocal()
    inventory_records = session.query(Inventory).all()
    order_records = session.query(Order).all()
    session.close()
    return inventory_records, order_records

inventory, order = getDbContent()

2025-08-18 14:00:29,939 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-08-18 14:00:29,940 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-18 14:00:29,944 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-08-18 14:00:29,945 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-18 14:00:29,946 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-08-18 14:00:29,946 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-18 14:00:29,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-18 14:00:29,949 INFO sqlalchemy.engine.Engine SELECT `Inventory`.`ItemId` AS `Inventory_ItemId`, `Inventory`.`ItemName` AS `Inventory_ItemName`, `Inventory`.`ItemCategory` AS `Inventory_ItemCategory`, `Inventory`.`ItemQuantity` AS `Inventory_ItemQuantity`, `Inventory`.`UnitsSold` AS `Inventory_UnitsSold`, `Inventory`.`Weight` AS `Inventory_Weight`, `Inventory`.`Size` AS `Inventory_Size`, `Inventory`.`Priority` AS `Inventory_Priority`, `Inventory`.`Location` AS `Inventory_Location`, `In

In [12]:
def dbtoList(records):
    output_list = []
    for r in records:
        if isinstance(r, Inventory):
            data = {
                "ItemId": r.ItemId, 
                "ItemName": r.ItemName,
                "Category": r.ItemCategory,
                "Quantity": r.ItemQuantity, 
                "UnitsSold": r.UnitsSold,
                "Weight": r.Weight, 
                "Size": r.Size,
                "Priority": r.Priority, 
                "Location": r.Location,
                "Date": r.Date, 
                "Dispose": r.Dispose                
            }
        elif isinstance(r, Order):
            data = {
                "OrderId": r.OrderId,
                "ItemId": r.ItemId, 
                "OrderQuantity": r.OrderQuantity, 
                "Sales": r.Sales, 
                "Price": r.Price, 
                "Discount": r.Discount,
                "Profit": r.Profit, 
                "DateOrdered": r.DateOrdered,
                "DateReceived": r.DateReceived,
                "CustomerSegment": r.CustomerSegment
            }
        else:
            continue
        output_list.append(data)
    
    return output_list

inventoryData = dbtoList(inventory)
orderData = dbtoList(order)

## Supervised Models

In [15]:
import pickle
import joblib
from datetime import datetime
from IPython.display import Markdown

class Supervised_Models:
    # Location Prediction Model
    def predict_location(input_data):
        with open('../../Supervised models/Samuel/storage_prediction_model.pkl', 'rb') as file:
            storage_prediction_model = pickle.load(file)
            
        categorical_features = {
            'Priority': ['High','Low','Medium'],
            'Product_Type': ['Clothing','Technology','Other','Sports and Fitness'],
            'Size': ['Large','Medium','Small']
        }
        numerical_features = ['Order_Quantity', 'Weight']
        one_hot_columns = []
        
        for feature, values in categorical_features.items():
            for value in values:
                one_hot_columns.append(f"{feature}_{value}")
            
        # Combine with numerical features to get all feature names
        all_feature_names = one_hot_columns + numerical_features

        features_dict = {col: 0 for col in all_feature_names}
    
        # Set one-hot encoded features
        for feature, values in categorical_features.items():
            if feature in input_data:
                selected_value = input_data[feature]
                one_hot_col = f"{feature}_{selected_value}"
                if one_hot_col in features_dict:
                    features_dict[one_hot_col] = 1
    
        # Set numerical features
        for feature in numerical_features:
            if feature in input_data:
                features_dict[feature] = float(input_data[feature])
        
        # Convert to array in the correct order
        features_array = np.array([features_dict[col] for col in all_feature_names]).reshape(1, -1)

        prediction = storage_prediction_model.predict(features_array)
        return prediction

    def demand_forecast_preprocessor(order_data, inventory_data):
        # Create Dataframe
        order = pd.DataFrame(order_data)
        inventory = pd.DataFrame(inventory_data)

        # Ensure dates are in datetime format
        order['DateOrdered'] = pd.to_datetime(order['DateOrdered'])
        
        # Extract Month (period or string, depending on preference)
        order['OrderMonth'] = order['DateOrdered'].dt.to_period('M')
        
        # Merge with Category lookup table
        merged_df = order.merge(inventory, on='ItemId', how='left')
        
        # Group and aggregate
        result_df = (
            merged_df
            .groupby(['OrderMonth', 'Category', 'CustomerSegment'], as_index=False)
            .agg(
                AveragePrice=('Price', 'mean'),
                AverageDiscount=('Discount', 'mean')
            )
        )
        
        return result_df

    # Demand forecast model
    def predict_demand_forecast(input_data):
        demand_forecast_model = joblib.load('../../Supervised models/ShernFai/model/salesforecast(categories).pkl')
        with open('../../Supervised models/ShernFai/model/salesforecast_preprocessor.pkl', 'rb') as f:
            preprocessor_data = pickle.load(f)

        categories = {
            "Clothing" : [
                "Cleats",
                "Men's Footwear",
                "Women's Apparel"
            ],
            "Technology": [
                "Electronics",
                "Video Games",
                "Cameras",
                "Computers",
            ],
            "Sports and Fitness": [
                "Cardio Equipment",
                "Indoor/Outdoor Games",
                "Water Sports",
                "Shop By Sport",
                "Camping & Hiking",
                "Fishing"
            ],
            "Other": [
                "Garden",
                "Pet Supplies"
            ]
        }

        cat_keys = list(categories.keys())

        # Extract preprocessor components
        le_category = preprocessor_data['label_encoder_category']
        reference_date = preprocessor_data['reference_date']
        unique_categories = preprocessor_data['unique_categories']
        feature_columns = preprocessor_data['feature_columns']

        # Get data
        category_name = input_data['category']
        future_month = input_data['month']
        avg_price = float(input_data['avg_price'])
        customer_segment = input_data['customer_segment']
        discount_rate = float(input_data['discount_rate'])
        
        # Parse the future date
        future_date = pd.to_datetime(future_month)
        
        # Calculate time features for the future date
        months_since_start = ((future_date - reference_date).days / 30.44)
        
        # Create test data with numerical time features
        test_data = {
            'Category Name': category_name,
            'Average Product Price': avg_price,
            'Customer Segment': customer_segment,
            'Order Item Discount Rate': discount_rate,
            # Time features (numerical - can handle ANY future date!)
            'Year': future_date.year,
            'Month': future_date.month,
            'Quarter': future_date.quarter,
            'Months_Since_Start': int(months_since_start),
            'Month_Sin': np.sin(2 * np.pi * future_date.month / 12),
            'Month_Cos': np.cos(2 * np.pi * future_date.month / 12),
            'Year_Trend': future_date.year - reference_date.year
        }
        
        # Create DataFrame
        test_df = pd.DataFrame([test_data])
        
        # Handle unknown category
        if category_name not in cat_keys:
            print(f"Unknown category '{category_name}' - using default: {cat_keys[0]}")
            test_df['Category Name'] = cat_keys[0]
            category_name = cat_keys[0]
        
        # One-hot encode customer segment
        test_df = pd.get_dummies(test_df, columns=['Customer Segment'], drop_first=True)
        
        # Ensure same columns as training (crucial!)
        test_df = test_df.reindex(columns=feature_columns, fill_value=0)
        
        # Make prediction
        total = 0
        num = len(categories[category_name])
        for subclass in categories[category_name]:
            test_df['Category Name'] = subclass
            test_df['Category Name'] = le_category.transform(test_df['Category Name'])
            total += demand_forecast_model.predict(test_df)
        
        avg_demand = total / num
        
        return avg_demand

    def detect_anomalies(inventory_list):
        anomalies_detected = []
        for item in inventory_list:
            current_location = item['Location']
            predicted_location = Supervised_Models.predict_location({
                "Priority": item['Priority'],
                "Product_Type": item['Category'],
                "Size": item['Size'],
                "Order_Quantity": item['Quantity'],
                "Weight": item['Weight']
            })[0]

            #print(f"\nCurrent Location: {current_location}")
            #print(f"Predicted Location: {predicted_location}")
            if current_location != predicted_location:
                #print(f"Anomaly detected! Item id:{item['ItemId']} is stored at location {current_location} while it should be stored at {predicted_location}.")
                anomalies_detected.append({'ItemId': item['ItemId'], 'ItemName': item['ItemName'], 'CurrentLocation': current_location, 'PredictedLocation': predicted_location})

        return anomalies_detected

s = Supervised_Models

# Testing

print(s.predict_location({
    "Priority": "Medium",
    "Product_Type": "Sports and Fitness",
    "Size": "Medium",
    "Order_Quantity": 12,
    "Weight": 10.78
}))

print(s.detect_anomalies(inventoryData))

result = s.demand_forecast_preprocessor(orderData, inventoryData)
result

print(s.predict_demand_forecast({
    'category': "Clothing",
    'month': "2025-05",
    'avg_price': 10.0,
    'customer_segment': "Consumer",
    'discount_rate': 0.12
}))

# Sections

## Products Overview

In [ ]:
# Product Overview: This section summarizes the overall performance of products
product_overview = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate an overview of products based on the following:

1. **Top Products**: Identify the top-selling products by sales volume and revenue over the past month.
2. **Product Performance**: Analyze the performance of each product category in terms of sales, demand, and revenue.
3. **Sales by Product**: Present a summary of sales for each product, including revenue, volume sold, and average price.

Data:
- Sales Data: {sales_data}
- Product Categories: {product_categories_data}
- Product Sales Volume: {sales_volume_data}
- Product Revenue: {product_revenue_data}
'''
)

display(Markdown(product_overview.text))


## Category Distribution

In [ ]:
# Category Distribution: This section analyzes the distribution of sales across categories
category_distribution = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate insights on the distribution of sales across categories based on the following:

1. **Category-wise Sales Volume**: Summarize the total sales volume across all product categories.
2. **Category-wise Revenue**: Display the total revenue generated from each product category over the past month.
3. **Category-wise Performance**: Provide insights on which product categories are performing the best in terms of sales, revenue, and customer demand.

Data:
- Sales Volume by Category: {sales_volume_by_category_data}
- Revenue by Category: {revenue_by_category_data}
- Product Category Performance: {category_performance_data}
'''
)

display(Markdown(category_distribution.text))


## Product Usage Forecast

In [ ]:
# Product Usage Forecast: This section forecasts the future usage of products based on historical sales
product_usage_forecast = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate a product usage forecast for the upcoming period based on historical data and trends:

1. **Sales Forecast by Product**: Predict the sales volume for each product in the next quarter/year.
2. **Demand Forecast**: Provide a demand forecast for products based on historical sales, usage probabilities, and seasonal patterns.
3. **Future Stock Levels**: Estimate the required stock levels for each product to meet forecasted demand.

Data:
- Historical Sales Data: {historical_sales_data}
- Product Usage Data: {usage_probabilities}
- Seasonal Patterns: {seasonal_sales_patterns_data}
- Current Stock Levels: {current_inventory_data}
'''
)

display(Markdown(product_usage_forecast.text))


## Sales Insights

In [99]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Sales Data
sales_data = orderData

# Sales Predictions
current_date = datetime.now()
next_month_date = current_date + relativedelta(months=1)
next_month_yearmonth = next_month_date.strftime("%Y-%m")

sales_pred_input = s.demand_forecast_preprocessor(orderData, inventoryData)
sales_predictions = []

for index, row in sales_pred_input.iterrows():
    sales_predictions.append({
        'Category Name': row.Category,
        'Customer Segment': row.CustomerSegment,
        'Predicted Demand for next month': s.predict_demand_forecast({
            'category': row.Category,
            'month': next_month_yearmonth,
            'avg_price': row.AveragePrice,
            'customer_segment': row.CustomerSegment,
            'discount_rate': row.AverageDiscount
        })[0]
    })

# Product Categories
product_categories = ["Clothing","Technology","Sports and Fitness","Other"]

# Current Inventory
current_inventory = inventoryData

# Usage Probabilities
usage_probabilities = "Currently empty. Please ignore this section for now."

D:\Yong Meng Lee\anaconda3\envs\it3100\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\Yong Meng Lee\anaconda3\envs\it3100\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [101]:
# Sales Insights Section
section_sales_insights = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate a sales insights report that describes information for the following areas:

1. Sales Trends: Summarize sales based on the provided data. Provide insights on which product categories are seeing the highest demand.
2. Product Performance: Analyze the best-selling product categories by quantity. Highlight the top 3 performing categories.
3. Product Demand Forecast: Based on the historical sales and usage probability, forecast the demand for the next month.
4. Restocking or Discontinuation: Recommend which products should be restocked and which should be discontinued, based on sales trends and inventory levels.

Data:
- Historical sales data: {sales_data}
- Sales volume predictions for next month: {sales_predictions}
- Product categories: {product_categories}
- Current inventory levels: {current_inventory}
- Usage probabilities: {usage_probabilities}'''
)

display(Markdown(section_sales_insights.text))

## Sales Insights Report: June - July 2025 Performance & Future Outlook

This report provides a comprehensive analysis of sales performance for June and July 2025, offering insights into sales trends, product performance, demand forecasts, and recommendations for inventory management.

---

### 1. Sales Trends

**Overall Sales Summary:**
Based on the provided historical data for June and July 2025, the total sales amounted to **$90,000** across 12 orders, with a total of **900 units** sold.

**Product Category Demand:**
The sales data indicates demand across two primary product categories: 'Technology' and 'Other'.

*   **Other Category:** This category is seeing the highest demand, generating **$60,000** in sales from **600 units** sold (e.g., Chairs). Sales for this category occurred consistently in July 2025.
*   **Technology Category:** This category generated **$30,000** in sales from **300 units** sold (e.g., Laptops). Sales for this category occurred consistently in June 2025.

**Insight:** While both categories contribute significantly, the 'Other' category currently holds a stronger position in terms of both total sales revenue and units sold within the recorded period.

---

### 2. Product Performance

Analyzing the best-selling product categories by quantity sold further highlights the current market preferences.

**Top Performing Categories (by Quantity Sold):**

1.  **Other:** 600 units sold (e.g., Chairs)
2.  **Technology:** 300 units sold (e.g., Laptops)

**Insight:** The 'Other' category demonstrates superior performance in terms of sales volume, selling twice as many units as the 'Technology' category within the observed timeframe. Given only two active product categories in the sales data, these are the top two.

---

### 3. Product Demand Forecast (Next Month)

Based on the provided sales volume predictions for the next month, here is the anticipated demand:

*   **Technology Category (Corporate Segment):** Forecasted demand of **126.70 units**.
*   **Other Category (Consumer Segment):** Forecasted demand of **128.16 units**.

**Insight:** The forecast suggests a relatively balanced demand for both categories in the upcoming month, with slightly higher demand for the 'Other' category. It's important to note that these predictions are tied to specific customer segments (Corporate for Technology, Consumer for Other).

---

### 4. Restocking or Discontinuation Recommendations

To provide recommendations, we will compare historical sales, predicted demand, and current inventory levels.

**Current Inventory Overview:**

*   **Technology (Laptops - ItemId 1):**
    *   Current Stock: 600 units (6 records x 100 units/record)
    *   Historical Units Sold (June-July): 300 units
    *   Predicted Demand (Next Month): ~127 units
*   **Other (Chairs - ItemId 2):**
    *   Current Stock: 1200 units (6 records x 200 units/record)
    *   Historical Units Sold (June-July): 600 units
    *   Predicted Demand (Next Month): ~128 units

**Recommendations:**

*   **Technology Category (Laptops):**
    *   **Recommendation: No immediate restocking required.**
    *   **Justification:** The current inventory of 600 units significantly exceeds both the historical sales volume (300 units over two months) and the predicted demand for the next month (~127 units). At the current sales rate, existing stock appears sufficient for at least 4-5 months, assuming consistent demand.
*   **Other Category (Chairs):**
    *   **Recommendation: No immediate restocking required.**
    *   **Justification:** Similarly, the current inventory of 1200 units is well above both historical sales (600 units over two months) and the predicted demand for the next month (~128 units). This stock level should comfortably cover demand for an extended period.
*   **Discontinuation:**
    *   **Recommendation: No products should be discontinued at this time.**
    *   **Justification:** Both 'Technology' and 'Other' categories are actively selling, showing consistent historical demand, and have positive demand forecasts for the next month. There are no indicators (e.g., zero sales, high inventory with no demand, or 'Dispose' flag being true) to suggest discontinuation for any listed item.

---

**Summary Conclusion:**

The business is experiencing healthy demand for both its 'Technology' and 'Other' product categories, with 'Other' currently leading in sales volume. Inventory levels for both categories are robust, indicating no immediate need for restocking. Continuous monitoring of sales trends and demand forecasts is recommended to optimize inventory strategy and capitalize on market opportunities.

## Storage Optimizations

In [105]:
location_predictions = []
for item in inventoryData:
    location_predictions.append({
        'Item Id': item['ItemId'],
        'Current Location': item['Location'],
        'Predicted Location': s.predict_location({
            'Priority': item['Priority'],
            'Product_Type': item['Category'],
            'Size': item['Size'],
            'Order_Quantity': item['Quantity'],
            'Weight': item['Weight']
        })[0]
    })

section_storage_optimizations = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Provide detailed storage optimization recommendations based on:

1. Current storage utilization metrics
2. Model-predicted optimal locations vs current locations
3. List of items flagged for relocation, including:
   - Current location
   - Recommended location

Data:
{inventoryData}
{location_predictions}
'''
)

display(Markdown(section_storage_optimizations.text))

This report provides detailed storage optimization recommendations based on your provided inventory data and model-predicted optimal locations.

## Executive Summary

The analysis reveals that your current inventory is split between two primary locations, A-1 and B-2. The model strongly recommends consolidating all items from both A-1 and B-2 into a single location, B-5. This suggests B-5 is either a new, highly optimized, more central, or higher-capacity storage area designed for improved efficiency. The primary recommendation is a full-scale relocation of all current stock to B-5, followed by a strategic re-evaluation of locations A-1 and B-2.

---

## 1. Current Storage Utilization Metrics

Based on the provided inventory data, here's an overview of the current storage utilization:

**Overall Inventory Snapshot:**

*   **Total Unique Items:** 12
*   **Total Quantity of Stock:** 1,800 units
    *   900 units of 'Laptop' (or similar Technology items)
    *   900 units of 'Chair' (or similar Other items)
*   **Total Estimated Volume Occupied:** 27,000 cubic units (assuming 'Size' is in cubic units per item)
*   **Total Estimated Weight:** 2,700 kg (assuming 'Weight' is per item)

**Utilization by Current Location:**

| Location | Item Category | Item Name (Examples) | Total Quantity | Total Volume (Est.) | Total Weight (Est.) |
| :------- | :------------ | :------------------- | :------------- | :------------------ | :------------------ |
| **A-1**  | Technology    | Laptop, Unknown Item | 900            | 9,000               | 1,350               |
| **B-2**  | Other         | Chair, Unknown Item  | 900            | 18,000              | 1,350               |
| **Total**|               |                      | **1,800**      | **27,000**          | **2,700**           |

**Observations:**

*   Locations A-1 and B-2 currently hold an equal number of total units (900 each) and total weight.
*   Location B-2, however, holds twice the estimated volume due to the larger size of 'Chair' units (20.0 vs 10.0 for 'Laptop').
*   Both 'Laptop' and 'Chair' items have a 50% turnover rate based on `UnitsSold / Quantity` (50/100 for Laptop, 100/200 for Chair), indicating they are relatively fast-moving items that benefit from accessible locations.
*   Items 9 and 10 have `ItemName` as `None`. While their physical attributes are recorded, their lack of a descriptive name should be addressed for better inventory management.
*   All items are marked `Dispose: False`, so no disposal recommendations are immediately required.

---

## 2. Model-Predicted Optimal Locations vs. Current Locations

The model's predictions are remarkably consistent and point towards a significant shift in your storage strategy.

**Key Findings:**

*   **Universal Recommendation for B-5:** For every single item (1 through 12), regardless of its current location (A-1 or B-2), the model recommends relocation to **B-5**.
*   **Zero Retention in Current Locations:** The model does not suggest keeping any items in A-1 or B-2, indicating these locations are suboptimal for your current inventory strategy.

**Interpretation & Implications:**

This uniform recommendation strongly suggests that **B-5 is considered the most efficient or suitable storage location for all your current inventory.** Possible reasons for B-5's optimality include:

*   **Strategic Consolidation:** B-5 might be a new central hub, closer to shipping/receiving, or optimized for faster picking and packing.
*   **Increased Efficiency:** It could incorporate automation, better layout, or specialized storage solutions that reduce retrieval times and operational costs.
*   **Capacity:** B-5 is presumed to have sufficient capacity to accommodate the entirety of your current inventory (1,800 units, 27,000 volume units).
*   **Product Characteristics Alignment:** B-5 might be ideally suited for both high-priority (Laptop) and larger-volume (Chair) fast-moving items due to its design or accessibility.

---

## 3. List of Items Flagged for Relocation

Based on the model's recommendations, **all 12 items currently in stock are flagged for relocation.**

| Item ID | Item Name     | Category   | Current Location | Recommended Location | Quantity | Priority | Units Sold (for context) |
| :------ | :------------ | :--------- | :--------------- | :------------------- | :------- | :------- | :----------------------- |
| 1       | Laptop        | Technology | A-1              | B-5                  | 100      | High     | 50                       |
| 2       | Chair         | Other      | B-2              | B-5                  | 200      | Medium   | 100                      |
| 3       | Laptop        | Technology | A-1              | B-5                  | 100      | High     | 50                       |
| 4       | Chair         | Other      | B-2              | B-5                  | 200      | Medium   | 100                      |
| 5       | Laptop        | Technology | A-1              | B-5                  | 100      | High     | 50                       |
| 6       | Chair         | Other      | B-2              | B-5                  | 200      | Medium   | 100                      |
| 7       | Laptop        | Technology | A-1              | B-5                  | 100      | High     | 50                       |
| 8       | Chair         | Other      | B-2              | B-5                  | 200      | Medium   | 100                      |
| 9       | Unknown Item  | Technology | A-1              | B-5                  | 100      | High     | 50                       |
| 10      | Unknown Item  | Other      | B-2              | B-5                  | 200      | Medium   | 100                      |
| 11      | Laptop        | Technology | A-1              | B-5                  | 100      | High     | 50                       |
| 12      | Chair         | Other      | B-2              | B-5                  | 200      | Medium   | 100                      |

---

## Detailed Storage Optimization Recommendations

Based on the data and model insights, here are the detailed recommendations:

### A. Immediate Action: Full Consolidation to B-5

1.  **Execute the Relocation Plan:** Systematically move all inventory from A-1 and B-2 to B-5 as per the "List of Items Flagged for Relocation" above.
    *   **Prioritization:** Consider prioritizing the relocation of 'High' priority items (Laptops, Items 1,3,5,7,9,11) first, as their accessibility is critical. However, given the universal recommendation, moving items by current location (e.g., clear A-1 then B-2) might be logistically simpler if B-5 has general purpose slots.
    *   **Batching:** Group similar items or items from the same current location for efficient movement.
    *   **Minimize Disruption:** Plan the relocation during off-peak hours or in phases to minimize impact on order fulfillment.

2.  **Optimize Slotting within B-5:** Once items arrive at B-5, don't just put them anywhere. Apply best practices:
    *   **Velocity-Based Slotting:** Since both Laptops and Chairs are relatively fast-moving, place them in the most accessible and frequently picked areas within B-5.
    *   **Size/Weight Considerations:** Place heavier/bulkier items (Chairs) on lower shelves for ergonomic picking and safety. Laptops can occupy mid-level shelves.
    *   **Category Grouping:** If B-5 is large enough, consider grouping items by category ('Technology', 'Other') to streamline future picking paths for multi-item orders.
    *   **High-Priority Zones:** If B-5 has designated "hot zones" or automated retrieval, ensure 'High' priority items (Laptops) are placed there.

### B. Strategic Re-evaluation of Locations A-1 and B-2

1.  **Evaluate for Decommissioning/Re-purposing:** With all inventory moved out, A-1 and B-2 will become empty.
    *   **Capacity Needs:** Determine if you anticipate future inventory growth that would necessitate reactivating these locations.
    *   **Alternative Uses:** Could A-1 and B-2 be re-purposed for:
        *   Returns processing areas?
        *   Kitting or assembly zones?
        *   Temporary overflow for peak seasons?
        *   Staging areas for outbound shipments?
        *   Equipment storage?
    *   **Cost Analysis:** If no immediate re-purpose is identified, consider the cost savings of closing or reducing operations in these areas (e.g., lighting, HVAC, maintenance).

2.  **Infrastructure Assessment:** While empty, this is an ideal time to conduct maintenance, upgrades, or safety checks on the shelving, flooring, and other infrastructure in A-1 and B-2.

### C. Continuous Improvement & Data Management

1.  **Understand the Model's Logic:** Gain deeper insights into *why* the model consistently recommends B-5. What criteria (e.g., item velocity, size, weight, priority, cost of access, available capacity) does it prioritize? This understanding is crucial for:
    *   **Future Slotting:** Applying the same logic for new incoming inventory.
    *   **Warehouse Design:** Informing future expansions or redesigns of other locations.
    *   **Model Validation:** Ensuring the model's recommendations align with operational realities and deliver desired efficiency gains.

2.  **Improve Data Quality:**
    *   **Resolve Missing Item Names:** Investigate and update the `ItemName` for `ItemId` 9 and 10 to ensure complete and accurate inventory records.
    *   **Standardize Data Entry:** Implement procedures to prevent future occurrences of missing or inconsistent data points.

3.  **Monitor Performance Post-Relocation:**
    *   **Key Performance Indicators (KPIs):** Track metrics such as:
        *   **Picking Efficiency:** Time taken per pick, picks per hour.
        *   **Order Fulfillment Cycle Time:** From order placement to dispatch.
        *   **Space Utilization in B-5:** Ensure the new location isn't becoming over-densified.
        *   **Labor Costs:** Reductions in travel time within the warehouse.
        *   **Inventory Accuracy:** Maintain high levels of accuracy in B-5.
    *   **Feedback Loop:** Use these KPIs to validate the success of the optimization and provide feedback to further refine the model or operational processes.

4.  **Consider Item Lifecycle Management:**
    *   While all items currently have `Dispose: False`, establish clear criteria and processes for identifying and managing slow-moving, obsolete, or damaged inventory in the future. This prevents dead stock from occupying valuable space.

---

By implementing these recommendations, you can significantly enhance your storage efficiency, streamline operations, and potentially reduce operational costs by leveraging the optimized capabilities of location B-5.

## Anomalies Detected

In [107]:
section_anomalies_detected = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate an anomalies section that lists all detected storage anomalies detected in a table. Include each item's current location, predicted location, item id, and name.
Include the reason for each anomaly.

Data:
{s.detect_anomalies(inventoryData)}
'''
)

display(Markdown(section_anomalies_detected.text))

## Storage Anomalies Detected

The following anomalies have been identified in the storage table:

### 1. Location Mismatches
These items are not in their predicted storage locations, indicating a potential misplacement or pending relocation.

*   **Item ID:** 1
    **Name:** Laptop
    **Current Location:** A-1
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 2
    **Name:** Chair
    **Current Location:** B-2
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 3
    **Name:** Laptop
    **Current Location:** A-1
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 4
    **Name:** Chair
    **Current Location:** B-2
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 5
    **Name:** Laptop
    **Current Location:** A-1
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 6
    **Name:** Chair
    **Current Location:** B-2
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 7
    **Name:** Laptop
    **Current Location:** A-1
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 8
    **Name:** Chair
    **Current Location:** B-2
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 9
    **Name:** [N/A]
    **Current Location:** A-1
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 10
    **Name:** [N/A]
    **Current Location:** B-2
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 11
    **Name:** Laptop
    **Current Location:** A-1
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

*   **Item ID:** 12
    **Name:** Chair
    **Current Location:** B-2
    **Predicted Location:** B-5
    **Reason:** The item's current location does not match its predicted storage location.

### 2. Missing Item Information
These items are missing crucial identification details.

*   **Item ID:** 9
    **Name:** [N/A]
    **Current Location:** A-1
    **Predicted Location:** B-5
    **Reason:** The item's name is missing, making identification difficult.

*   **Item ID:** 10
    **Name:** [N/A]
    **Current Location:** B-2
    **Predicted Location:** B-5
    **Reason:** The item's name is missing, making identification difficult.

## Summary

# PDF File Generation

In [110]:
from markdown_pdf import MarkdownPdf, Section

# Read Markdown content
mdc = f'''<h1 style="text-align:center;">Monthly Report</h1><br>

# Products Overview:
section_products_overview.text

# Category Distribution:
section_category_distribution.text

# Product Usage Forecast:
section_product_usage.text

# Sales Insights:
{section_sales_insights.text}

# Storage Optimizations:
{section_storage_optimizations.text}

# Anomalies Detected:
{section_anomalies_detected.text}

# Summary:
section_summary.text
'''
pdf = MarkdownPdf()
pdf.add_section(Section(mdc)) # Add Section(md_content, user_css=css_content) for custom CSS
pdf.save("MonthlyReport.pdf")